# Introduction
___

## Notebook Outline
    1. Updating Python dependencies
    2. Loading our data from an Excel file
    3. Creating the basic volcano plot with Bokeh and NumPy
    4. Coloring our data points based on criteria
    5. Adding our cutoff lines
    6. Adding interactivity to our volcano plot
    7. Multiple hypothesis correction
    8. Cleaning up and saving the figure as an image file 


## Follow along

You may follow along on your computer by cloning the [PGSRM-2019-Workshop repository](https://github.com/ijmiller2/PGSRM-2019-Workshop "2019-PGSRM-workshop") from GitHub and opening up the jupyter notebook page using [jupyter notebooks](https://jupyter.org/ "Jupyter Notebooks Homepage")

```bash
git clone https://github.com/ijmiller2/PGSRM-2019-Workshop.git
jupyter notebook PGSRM-2019-Workshop
```

__or__

by following this link: https://notebooks.azure.com/ijmiller2/projects/pgsrm-2019-workshop

# Updating Python dependencies
---

If you have Anconda Python, you can update your Python Packages with `pip`. For this exercise, we'll need the latest version of [Bokeh](http://bokeh.pydata.org/en/latest/ "Bokeh Docs Homepage"), an interactive, web-based plotting library in Python. 

You can run Bash commands in Jupyter Notebooks using the exclamation point as an "escape character."

In [ ]:
# upgrade bokeh with pip package manager
!pip install --upgrade bokeh

## Loading our data from an Excel file

We will load our data with [pandas](https://pandas.pydata.org/ "Pandas homepage"). This is short for **pan**eled **da**taframe**s** and is quite useful for manipulating tabular files.

In [ ]:
# load data with pandas
import pandas as pd

path_to_data = "../data/elife-31098-supp1-v2.xls"
df = pd.read_excel(path_to_data)

In [ ]:
# inspect data with head method
df.head()

# Creating the volcano plot
___
## Bokeh

We will use [Bokeh](http://bokeh.pydata.org/en/latest/ "Bokeh Docs Homepage") to recreate our figure. This is a python library containing functions to construct interactive visualizations.

The next cell will import all of the required functions to start constructing our figure.

In [ ]:
# load bokeh dependencies for the volcano plot 
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.io import output_notebook
# set up notebok environment to render bokeh plots inline
output_notebook()

## NumPy

We can use other existing python tools to perform our calculations. In this case, we will use a function from the [NumPy](https://www.numpy.org/ "Numpy homepage") library to determine the log transform for each of our data points.

In [ ]:
# we'll need numpy for the log transform
import numpy as np

## Adding data to a figure object

In [ ]:
# initialize figure object, estimating dimensions and axis ranges
plot = figure(plot_width=400, plot_height=500, 
              x_range=(-6,6), y_range=(0,16))

# add the points as a circle glyph
plot.circle(x=df['log2 Fold Change (H/L) (KRASG12V/Empty Vector)'],
         y=-np.log10(df['p-value']))

# format the axis labels
plot.xaxis.axis_label = "log2(H/L) (KRAS-G12V/EV)" #TODO: Include conditions
plot.yaxis.axis_label = "-log10(p-value)"

# render the plot
show(plot)

## Coloring our data points based on criteria

### Adding another column to our dataframe

In [ ]:
# initialize an empty list to store the color information
color_list = []

# iterate through the pandas dataframe and assign a color based on the criteria
for index, row in df.iterrows():
    
    FoldChange = row['log2 Fold Change (H/L) (KRASG12V/Empty Vector)']
    p_val = row['p-value']
    
    # significantly upregulated proteins
    if FoldChange >= 1 and p_val <= 0.01:
        color_list.append("blue")
    
    # significantly downregulated proteins
    elif FoldChange <= -1 and p_val <= 0.01:
        color_list.append("red")
        
    # all other proteins 
    else:
        color_list.append("black")
    
# add this list to your dataframe and then inspect with head()
df['color'] = color_list
df.head(n=10)

### Constructing a figure with colors based on the criteria

In [ ]:
# initialize figure object, estimating dimensions and axis ranges
plot = figure(plot_width=400, plot_height=500, 
              x_range=(-6,6), y_range=(0,16))

# add the points as a circle glyph
plot.circle(
    x=df['log2 Fold Change (H/L) (KRASG12V/Empty Vector)'],
    y=-np.log10(df['p-value']),
    color=df['color'],
    line_color='black',
)

# format the axis labels
plot.xaxis.axis_label = "log2(H/L) (KRAS-G12V/EV)" #TODO: Include conditions
plot.yaxis.axis_label = "-log10(p-value)"

show(plot)

## Adding our cutoff lines

We will add horizontal and vertical cutoffs using [spans](http://bokeh.pydata.org/en/latest/docs/user_guide/annotations.html#spans "Bokeh Spans Documentation"). The Spans function will allow us to annotate our figure with a line of the specified dimensions.

In [ ]:
from bokeh.models import Span

# p-value cutoff of 0.01 --> -log10(0.01) = 2
hline = Span(
    location=2,
    dimension='width', 
    line_color='black', 
    line_width=1,
    line_dash='dashed',
)

# cutoff for a negative 2-fold change
vline_left = Span(
    location=-1,
    dimension='height', 
    line_color='black', 
    line_width=1,
    line_dash='dashed',
)

# cutoff for a positive 2-fold change
vline_right = Span(
    location=1,
    dimension='height', 
    line_color='black', 
    line_width=1,
    line_dash='dashed',
)

# update the figure object's attributes
plot.renderers.extend([hline, vline_left, vline_right])

# show the updated image
show(plot)

# Adding interactivity to our volcano plot
___

Bokeh [tooltips](https://bokeh.pydata.org/en/latest/docs/user_guide/tools.html#basic-tooltips "Bokeh Basic tooltips Documentation") allow the user to interact with the generated figure. In our case, we will display the gene name associated with each point. 

This will also require a different method for passing in our data. This will be performed via the [ColumnDataSource](https://bokeh.pydata.org/en/latest/docs/reference/models/sources.html#bokeh.models.sources.ColumnDataSource "Bokeh ColumnDataSource Documentation") function.

In [ ]:
from bokeh.models import ColumnDataSource


# prepare the source data object
source = ColumnDataSource(
    data=dict(
        x=df['log2 Fold Change (H/L) (KRASG12V/Empty Vector)'],
        y=-np.log10(df['p-value']),
        Gene=df['Gene'],
        color=df['color'],
        p_vals=df['p-value']
    )
)


# Define tooltips
TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ("Gene", "@Gene"),
    ("p-value", "@p_vals")
]

# initialize figure object, estimating dimensions and axis ranges
plot = figure(plot_width=400, plot_height=500, 
              x_range=(-6,6), y_range=(0,16),
              tooltips=TOOLTIPS)


# add the points as a circle glyph
plot.circle(
    x='x',
    y='y',
    color='color',
    line_color='black',
    source=source
)

# p-value cutoff of 0.01 --> -log10(0.01) = 2
hline = Span(
    location=2,
    dimension='width', 
    line_color='black', 
    line_width=1,
    line_dash='dashed',
)

# cutoff for a negative 2-fold change
vline_left = Span(
    location=-1,
    dimension='height', 
    line_color='black', 
    line_width=1,
    line_dash='dashed',
)

# cutoff for a positive 2-fold change
vline_right = Span(
    location=1,
    dimension='height', 
    line_color='black', 
    line_width=1,
    line_dash='dashed',
)

# update the figure object's attributes
plot.renderers.extend([hline, vline_left, vline_right])
        

show(plot)

# Multiple hypothesis correction

Because each point on the plot represents the result of a statistical hypothesis test (in this case, the Student's t-test), we may want to adjust our *p*-values accordingly. For more details on this rationale, refer to [John McDonald's Biostats Handbook](http://www.biostathandbook.com/multiplecomparisons.html).

In this case, we will adjust our using the Benjamini Hochberg procedure. This will require the python library, [statsmodels](https://www.statsmodels.org/stable/index.html "Statsmodels Homepage")

In [ ]:
# multiple hypothesis correction
from statsmodels.stats.multitest import multipletests

passes_multiple_testing = multipletests(df['p-value'], alpha=0.01, method='fdr_bh')[0]
corrected_p_vals = multipletests(df['p-value'], alpha=0.01, method='fdr_bh')[1]

df['passes_multiple_testing'] = passes_multiple_testing
df['adjusted_p-values'] = corrected_p_vals
df.head()

## Updating the tooltips with corrected *p*-values

In [ ]:
# update tool tips 
# add tooltips

from bokeh.models import ColumnDataSource

# prepare the source data object
source = ColumnDataSource(data=dict(

    x=df['log2 Fold Change (H/L) (KRASG12V/Empty Vector)'],
    y=-np.log10(df['p-value']),
    Gene=df['Gene'] ,
    color=df['color'],
    p_vals=df['p-value'],
    adjusted_p_vals=df['adjusted_p-values']

))

# Define tooltips
TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ("Gene", "@Gene"),
    ("p-value", "@p_vals"),
    ("adjusted p-values", "@adjusted_p_vals")

]

# initialize figure object, estimating dimensions and axis ranges
plot = figure(plot_width=400, plot_height=500, 
              x_range=(-6,6), y_range=(0,16),
              tooltips=TOOLTIPS)

# add the points as a circle glyph
plot.circle(x='x',
         y='y',
         color='color',
         line_color='black',
         source=source)

# format the axis labels
plot.xaxis.axis_label = "log2(H/L) (KRAS-G12V/EV)" #TODO: Include conditions
plot.yaxis.axis_label = "-log10(p-value)"

# update the figure object's attributes
plot.renderers.extend([hline,vline_left,vline_right])
        
show(plot)

## Cleaning up and saving the figure as an image file 

In [ ]:
# hide grid lines
plot.xgrid.visible = False
plot.ygrid.visible = False

# remove minor ticks
plot.xaxis.minor_tick_line_color = None
plot.yaxis.minor_tick_line_color = None

# remove border
plot.outline_line_color = None

# format and save plot as a vector graphic
plot.output_backend = "svg"
plot.toolbar.logo = None

show(plot)